In [1]:
!pip install pandas numpy scikit-learn mlflow==0.9.1

    100% |████████████████████████████████| 11.8MB 2.7MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 26.6MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 34.1MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 26.6MB/s ta 0:00:01
    100% |████████████████████████████████| 92kB 28.0MB/s ta 0:00:01
    100% |████████████████████████████████| 450kB 28.2MB/s ta 0:00:01
    100% |████████████████████████████████| 81kB 23.8MB/s ta 0:00:01
    100% |████████████████████████████████| 122kB 36.3MB/s ta 0:00:01
    100% |████████████████████████████████| 51kB 19.4MB/s ta 0:00:01
    100% |████████████████████████████████| 5.6MB 6.7MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 27.1MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 24.8MB/s ta 0:00:01
    100% |████████████████████████████████| 552kB 26.8MB/s ta 0:00:01


    100% |████████████████████████████████| 163kB 35.2MB/s ta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/ee/09/99/bf937e4f02788fa8b33dc5240842ba3977ba5c3c4ad4a115d7
  Stored in directory: /home/jovyan/.cache/pip/wheels/5d/1a/1e/0350bb3df3e74215cd91325344cc86c2c691f5306eb4d22c77
  Stored in directory: /home/jovyan/.cache/pip/wheels/bc/a5/8d/7d021f2741ed8a7354e4fe19c1890e71ed719d681cd16cf292
  Stored in directory: /home/jovyan/.cache/pip/wheels/2b/67/89/414471314a2d15de625d184d8be6d38a03ae1e983dbda91e84
  Stored in directory: /home/jovyan/.cache/pip/wheels/fe/b9/8f/764ed47b11e2e062ae8fe5f09d1c801a600292fdd9ba3477bb
Successfully built querystring-parser simplejson mleap tabulate nose-exclude
You are using pip version 19.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
# The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
# P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
# Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.

import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn


def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

    
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    users_home = '/mnt/pipelineai/users'
    experiment_base_path = '%s/experiments' % users_home
    tracking_uri='file://%s' % experiment_base_path
    mlflow.set_tracking_uri(tracking_uri)

    experiment_name = 'wine'

    mlflow.set_experiment(experiment_name)    

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    wine_path = './wine-quality.csv'
    data = pd.read_csv(wine_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]

    alpha = 0.5
    l1_ratio = 0.5

    with mlflow.start_run():
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        predicted_qualities = lr.predict(test_x)

        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(lr, "model")

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772689


# Check the MLflow Pipelines Tab
![MLflow Pipeline](https://raw.githubusercontent.com/PipelineAI/site/master/assets/img/mlflow-pipeline.png)